# Imports

In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf

# Load datasets

In [2]:
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')

train_targets_a = pd.read_parquet('../data/A/train_targets.parquet')
train_targets_b = pd.read_parquet('../data/B/train_targets.parquet')
train_targets_c = pd.read_parquet('../data/C/train_targets.parquet')

# Data clean up

researching which columns have the most null values. 

In [3]:
for column in X_train_estimated_a.columns:
    null_c =  X_train_estimated_a[column].isna().sum()
    if null_c > 0: 
        print(f'Column: {column} has {null_c} NULL values')

for column in X_train_observed_a.columns:
    null_c = X_train_observed_a[column].isna().sum()
    if null_c > 0: 
        print(f'Column: {column} has {null_c} NULL values')

Column: ceiling_height_agl:m has 3919 NULL values
Column: cloud_base_agl:m has 2094 NULL values
Column: snow_density:kgm3 has 15769 NULL values
Column: ceiling_height_agl:m has 22247 NULL values
Column: cloud_base_agl:m has 8066 NULL values
Column: snow_density:kgm3 has 115945 NULL values


Based on output we choose to drop `snow_density:kgm3`, `ceiling_height_agl:m`, `cloud_base_agl:m`

```
X_ESTIMATED
Column: ceiling_height_agl:m has 3919 NULL values
Column: cloud_base_agl:m has 2094 NULL values
Column: snow_density:kgm3 has 15769 NULL values

X_OBSERVED:
Column: ceiling_height_agl:m has 22247 NULL values
Column: cloud_base_agl:m has 8066 NULL values
Column: snow_density:kgm3 has 115945 NULL values

```

In [4]:
""" for column in X_train_estimated_a.columns:
    null_c =  X_train_estimated_a[column].value_counts()[0]
    if null_c > 0: 
        print(f'Column: {column} has {null_c} 0 values')

for column in X_train_observed_a.columns:
    null_c = X_train_observed_a[column].value_counts()[0]
    if null_c > 0: 
        print(f'Column: {column} has {null_c} 0 values') """

" for column in X_train_estimated_a.columns:\n    null_c =  X_train_estimated_a[column].value_counts()[0]\n    if null_c > 0: \n        print(f'Column: {column} has {null_c} 0 values')\n\nfor column in X_train_observed_a.columns:\n    null_c = X_train_observed_a[column].value_counts()[0]\n    if null_c > 0: \n        print(f'Column: {column} has {null_c} 0 values') "

In [5]:

# Data set A, B and C clean up

def data_clean_up(x_train_est, x_train_observe, y_train):

  if 'date_calc' in x_train_est.columns:
    x_train_est.drop(columns="date_calc", inplace=True)

  x_train = pd.concat([x_train_observe, x_train_est])

  # Group the rows into blocks of 4 and apply the aggregation function
  agg_func = {col: 'mean' for col in x_train.columns[1:]}
  X_train_downscaled = x_train.groupby(x_train.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})

  y_train.dropna(inplace=True)
  combined_data = pd.merge(X_train_downscaled, y_train, left_on='date_forecast', right_on='time')
  combined_data.drop(columns=['snow_density:kgm3', 'ceiling_height_agl:m', 'cloud_base_agl:m'], inplace=True)
  combined_data.dropna(inplace=True)
  y_train = combined_data[['pv_measurement', 'date_forecast']]

  if 'time' and 'pv_measurement' in combined_data.columns:
    combined_data.drop(columns=['time', 'pv_measurement'], inplace=True)
    combined_data.drop(columns="pv_measurement", inplace=True)

  return combined_data, y_train

def count_null_in_column(df: pd.DataFrame, column_name: str):
  return df[column_name].value_counts(None)

x_train_a, y_train_a = data_clean_up(X_train_estimated_a, X_train_observed_a, train_targets_a)
x_train_b, y_train_b = data_clean_up(X_train_estimated_b, X_train_observed_b, train_targets_b)
x_train_c, y_train_c = data_clean_up(X_train_estimated_c, X_train_observed_c, train_targets_c)


def data_clean_up_test(x_test_est):

  # Group the rows into blocks of 4 and apply the aggregation function
  agg_func = {col: 'mean' for col in x_test_est.columns[1:]}
  X_test_downscaled = x_test_est.groupby(x_test_est.index // 4).agg({**{'date_forecast': 'first'}, **agg_func})

  """
  if 'date_forecast' in X_test_downscaled.columns:
    X_test_downscaled.drop(columns="date_forecast", inplace=True)
  """
  return X_test_downscaled

X_test_estimated_a = data_clean_up_test(X_test_estimated_a)
X_test_estimated_b = data_clean_up_test(X_test_estimated_b)
X_test_estimated_c = data_clean_up_test(X_test_estimated_c)
print(len(X_test_estimated_a))



720


Keeping this average for now since its the best aggreagation we have at this point. However i do believe that using avg or mean together with delta (total change within the hour) might be a better solution. Also some columns might need other aggregations than avg. 

# Feature engineering

Need to tell the model something about how time seasonality works. E.g. night and day, as well as yearly seasonality. 

In [6]:
def get_unixtime(datetime: pd.Series) -> pd.Series:
    unixtime = (datetime - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
    return unixtime

xat_datetime = x_train_a['date_forecast']
xat_unixtime = get_unixtime(xat_datetime)

## We now need functions for assigning daily and yearly cycles (described in datanalysis docu on Peter branch)
# plus 2 avoids 0 and negative values
day = 24*60*60
year = (365.2425)*day

def sinus_day(unix_time):
    return 2 + np.sin(unix_time * (2 * np.pi / day)) # since it is seconds since 1.1.1970 we divide by seconds in a day to get seasonal changes throughout the dat

def sinus_year(unix_time):
    return 2+ np.sin(unix_time * (2 * np.pi / year))

def cosinus_day(unix_time):
    return 2+np.cos(unix_time * (2 * np.pi / day))

def cosinus_year(unix_time):
    return 2+np.cos(unix_time * (2 * np.pi / year))

# function for returning two series with the daily cycles (sine and cosine)
def get_daycycle(unixtime: pd.Series) -> (pd.Series, pd.Series):
    sinus_daytime = unixtime.apply(sinus_day)
    sinus_daytime = sinus_daytime.rename('sinus_day') 
    cosinus_daytime = unixtime.apply(cosinus_day)
    cosinus_daytime = cosinus_daytime.rename('cosine_day')
    return sinus_daytime, cosinus_daytime

# Function for returning two series with the yearly cycles
def get_yearcycle(unixtime: pd.Series) -> (pd.Series, pd.Series):
    sinus_yeartime = unixtime.apply(sinus_year)
    sinus_yeartime = sinus_yeartime.rename('sinus_year')
    cosinus_yeartime = unixtime.apply(cosinus_year)
    cosinus_yeartime = cosinus_yeartime.rename('cosinus_year')
    return sinus_yeartime, cosinus_yeartime

xat_day_sin, xat_day_cos = get_daycycle(xat_unixtime)
xat_year_sin, xat_year_cos = get_yearcycle(xat_unixtime)

xta_feat = x_train_a.join([xat_day_sin, xat_day_cos, xat_year_sin, xat_year_cos])
xta_feat.drop(columns=['date_forecast'], inplace=True)
print(xta_feat.head())

   absolute_humidity_2m:gm3  air_density_2m:kgm3  clear_sky_energy_1h:J  \
0                    8.0250             1.230625               0.000000   
1                    7.9000             1.228750               0.000000   
2                    8.0125             1.224750               0.000000   
3                    8.3125             1.223250             104.324997   
4                    8.6625             1.222500           16234.075195   

   clear_sky_rad:W  dew_or_rime:idx  dew_point_2m:K  diffuse_rad:W  \
0            0.000              0.5      280.787506         0.0000   
1            0.000              0.5      280.574982         0.0000   
2            0.000              0.5      280.787506         0.0000   
3            0.375              0.5      281.362488         0.1500   
4           11.550              0.5      281.924988         5.9875   

   diffuse_rad_1h:J  direct_rad:W  direct_rad_1h:J  ...  total_cloud_cover:p  \
0          0.000000         0.000         0.0000

Here we need to do some extra feature engineering with regards to different datatypes in the dataset. A lot of this is explained in the TensorFlow documentation https://www.tensorflow.org/tutorials/load_data/pandas_dataframe. 

### Setting categorical and binary values as int

Tensorflow usuallt interprets data as float32. However for categorical and binary data we want it to be interpreted as integers. Therefore we set these specific columns as datatype int, and the rest as datatype float. Here we utilize the dictionary way of creating a model as opposed to the numpy array way. This is because our data is NOT homogenous. 

In [7]:
target = y_train_a
target.drop(columns='date_forecast', inplace=True)
xta_numeric_dict_ds = tf.data.Dataset.from_tensor_slices((dict(xta_feat), target))
for row in xta_numeric_dict_ds.take(3):
  print(row)

({'absolute_humidity_2m:gm3': <tf.Tensor: shape=(), dtype=float32, numpy=8.025>, 'air_density_2m:kgm3': <tf.Tensor: shape=(), dtype=float32, numpy=1.230625>, 'clear_sky_energy_1h:J': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'clear_sky_rad:W': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'dew_or_rime:idx': <tf.Tensor: shape=(), dtype=float32, numpy=0.5>, 'dew_point_2m:K': <tf.Tensor: shape=(), dtype=float32, numpy=280.7875>, 'diffuse_rad:W': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'diffuse_rad_1h:J': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'direct_rad:W': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'direct_rad_1h:J': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'effective_cloud_cover:p': <tf.Tensor: shape=(), dtype=float32, numpy=99.5375>, 'elevation:m': <tf.Tensor: shape=(), dtype=float32, numpy=6.0>, 'fresh_snow_12h:cm': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'fresh_snow_1h:cm': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'fresh_sn

/var/folders/94/wm45lcfj7sv746_0rnyvdk840000gn/T/ipykernel_32375/1542623841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target.drop(columns='date_forecast', inplace=True)


### Preprocessing pipeline

In [8]:
binary_featurenames = ['is_day:idx', 'is_in_shadow:idx']
categorical_featurenames = ['dew_or_rime:idx', 'precip_type_5min:idx', 'snow_drift:idx']

inputs = {}
for name, column in xta_feat.items():
  if type(column[0]) == str:
    dtype = tf.string
  elif (name in categorical_featurenames or
        name in binary_featurenames):
    dtype = tf.int64
  else:
    dtype = tf.float32
  inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)
inputs

{'absolute_humidity_2m:gm3': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'absolute_humidity_2m:gm3')>,
 'air_density_2m:kgm3': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'air_density_2m:kgm3')>,
 'clear_sky_energy_1h:J': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'clear_sky_energy_1h:J')>,
 'clear_sky_rad:W': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'clear_sky_rad:W')>,
 'dew_or_rime:idx': <KerasTensor: shape=(None,) dtype=int64 (created by layer 'dew_or_rime:idx')>,
 'dew_point_2m:K': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'dew_point_2m:K')>,
 'diffuse_rad:W': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'diffuse_rad:W')>,
 'diffuse_rad_1h:J': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'diffuse_rad_1h:J')>,
 'direct_rad:W': <KerasTensor: shape=(None,) dtype=float32 (created by layer 'direct_rad:W')>,
 'direct_rad_1h:J': <KerasTensor: shape=(None,) dtype=float32 (cr

In [9]:
preprocessed = []

for name in binary_featurenames:
  inp = inputs[name]
  inp = inp[:, tf.newaxis]
  float_value = tf.cast(inp, tf.float32)
  preprocessed.append(float_value)

preprocessed

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tf.cast')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tf.cast_1')>]

In [11]:
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
      values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

# getting numerical features
numeric_feature_names = []
for col in xta_feat.columns:
   if col not in binary_featurenames and col not in categorical_featurenames:
    numeric_feature_names.append(col)
numeric_features = xta_feat[numeric_feature_names]

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(numeric_features)))

# creating new dict to send to normalizer and append to preprocessed
numeric_inputs = {}
for name in numeric_feature_names:
  numeric_inputs[name]=inputs[name]

numeric_inputs = stack_dict(numeric_inputs)
numeric_normalized = normalizer(numeric_inputs)

preprocessed.append(numeric_normalized)

preprocessed

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tf.cast')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tf.cast_1')>,
 <KerasTensor: shape=(None, 41) dtype=float32 (created by layer 'normalization_1')>]

In [12]:
for name in categorical_featurenames:
  vocab = sorted(set(xta_feat[name]))
  print(f'name: {name}')
  print(f'vocab: {vocab}\n')

  if type(vocab[0]) is str:
    lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot')
  else:
    lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, output_mode='one_hot')

  x = inputs[name][:, tf.newaxis]
  x = lookup(x)
  preprocessed.append(x)

name: dew_or_rime:idx
vocab: [-1.0, -0.75, -0.5, -0.375, -0.25, -0.125, 0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 1.0]



2023-10-09 13:34:30.167129: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at lookup_table_op.cc:1066 : FAILED_PRECONDITION: HashTable has different value for same key. Key 0 has 1 and trying to add value 2


FailedPreconditionError: {{function_node __wrapped__LookupTableImportV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} HashTable has different value for same key. Key 0 has 1 and trying to add value 2 [Op:LookupTableImportV2] name: 

# Training the model

In [ ]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

model.fit(dict(xta_feat), target, epochs=5, batch_size=BATCH_SIZE)


# Make predictions

In [ ]:
# Do some more stuff

# Evaluate prediction

In [ ]:
# Evaluate the model

y_pred = []

# Create submission

In [ ]:
y_test_pred = y_pred

test = pd.read_csv('../data/test.csv')
test['prediction'] = y_test_pred
sample_submission = pd.read_csv('../data/sample_submission.csv')
submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
submission.to_csv('submission.csv', index=False)